### Project Objective

Investigate whether there is any measurable difference in progress based on the eligibility category. 
There are three options: Preliminary Analysis

+ Developmental evaluation (delay)
+ Diagnosed condition
+ Diagnosed condition, developmental evaluation (by both delay and diagnosis)

1. Start by looking at overall progress by eligibility category (column AI of the "Elig Timeline Rpt 2018-2022" tab)
2. Factor in the time of service ("ECO with Exit-21-22" tab)
3. Do the above comparison by POE as well (column A of the "ECO with Exit21-22") Additional Analysis Additional areas you can look into are listed below:
+ Does typical time of service differ for different eligibility categories?
+ Do exit reasons vary by eligibility category? Do more children in one eligibility category age out compared to leaving for other reasons?
+ "ECO with Exit21-22" contains the entry ECO scores (columns D, E, and F) and exit scores (columns H, I, and J) Analyze these scores by looking at the typical improvement seen for each entry rating compared to the time of service. What percentage of children entering with a score of 1 also exit with a score of 1? How many improve to a 2 or a 3? How does this vary by time of service?

### Part 2

Does typical time of service differ for different eligibility categories?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
%matplotlib inline

In [ ]:
#reading in the data from the eco_exit sheet 
eco_21_22_exit = pd.read_excel("../Data/TEIS-NSS Project Data 10-2022.xlsx", sheet_name="ECO with Exit21-22", nrows=8632)
eco_21_22_exit

In [ ]:
#rename the columns so that i can join them later to the timeline data sheet
eco_21_22_exit.rename(columns = {'CHILD_ID':'Child ID'}, inplace = True)
eco_21_22_exit

In [ ]:
#reading in the second sheet we need to do the analysis
eco_21_22_data = pd.read_excel("../Data/TEIS-NSS Project Data 10-2022.xlsx", sheet_name="Elig Timeline Rpt 2018-2022")
eco_21_22_data

In [ ]:
#renaming the columns because the . does not work well when calling on it later 
eco_21_22_data.rename(columns = {'Init. Elig. Category': 'Init_Elig_Cat'}, inplace=True)
eco_21_22_data

In [ ]:
#making a new dataframe out the information in the timeline sheet that only has the two columns i am interested in 
df2_data = eco_21_22_data[['Child ID','Init_Elig_Cat']]
df2_data

In [ ]:
#making a new dataframe with only the columns I am interested in in the eco sheet
df1_exit = eco_21_22_exit[['Child ID','Days btw Initial and Exit', 'Days btw I-IFSP to Exit ECO', '<Calc> Entrance Age (months)']].dropna()
df1_exit

In [ ]:
#renamed the age column to be easier to call in later on graphs
df1_exit = df1_exit.rename(columns={'<Calc> Entrance Age (months)': 'Entrance Age'})
df1_exit

In [ ]:
#makng sure to exclude any data that's negative or not 6 at least 6 months 
df1_exit = df1_exit.loc[(df1_exit['Days btw Initial and Exit']>=183)]
df1_exit = df1_exit.loc[(df1_exit['Days btw I-IFSP to Exit ECO']>0)]

In [ ]:
#checking the dataframe now to make sure it got reid of those 
df1_exit

In [ ]:
#now i am merging the two datafames together on child id
df_merge1 = df2_data.merge(df1_exit, on = 'Child ID')
df_merge1


In [ ]:
#i then thought it would be nice to look at the averages b/c the question wants to know time in program based on eligibilty category. Used a groupby function/call - you will get a warning - just ignore 
df1_merge_group = df_merge1.groupby('Init_Elig_Cat', as_index=False)['Days btw Initial and Exit','Days btw I-IFSP to Exit ECO', 'Entrance Age'].mean()
df1_merge_group

<b> CALLOUT: <b>
   + it's interesting the average age (by month) is all close to 3yr at the entrance/start into the program?

In [ ]:
#wanted to make columns in the dataframe that also gave me the avg months in the program in case that was interesting to see when making graphs - i also rounded it because i didn't think the ling line of numbers was nice looking  
df1_merge_group['Avg Months btw I and E'] = df1_merge_group['Days btw Initial and Exit']/30
df1_merge_group.round()

In [ ]:
#adding the second column 
df1_merge_group['Avg Months btw I-IFSP and E ECO'] = df1_merge_group['Days btw I-IFSP to Exit ECO']/30
df1_merge_group.round()

In [ ]:
#final column
df1_merge_group = df1_merge_group.round()
df1_merge_group

In [ ]:
#combines them together... 
#avg_tidy = df_merge1.melt(id_vars= '', var_name = '')
#avg_tidy

In [ ]:
#creating a new dataframe so i can try to make this plotly graph and not mess up anything 
moving_df1_merge_group = df1_merge_group.drop(columns = ['Days btw I-IFSP to Exit ECO', 'Entrance Age', 'Avg Months btw I and E', 'Avg Months btw I-IFSP and E ECO' ])
moving_df1_merge_group

In [ ]:
df2 = moving_df1_merge_group
df2

In [ ]:
# # pump the breaks on plotly!!
# # #trying out buttons 
# import plotly.graph_objects as px

# # #reading the data
# plot = px.Figure(df2=[px.Scatter(
#         x=df2['Init_Elig_Cat'],
#         y=df2['Days btw Initial and Exit'],
#         mode='markers')
#         ])

# # #add dropdown 
# plot.update_layout(
#             updatemenus=[
#                 dict(
#                     type="buttons",
#                     direction="left",
#                     buttons=list([
#                         dict(
#                             args=["type", "scatter"],
#                             label="Scatter Plot",
#                             method="restyle"
#                         ),
#                         dict(
#                             args=["type", "bar"],
#                             label="Bar Chart",
#                             method="restyle"
#                         )
                        
#                     ]),
#                 ),
                
#             ]
#  )

# plot.show()


In [ ]:
df2

In [ ]:

import plotly.express as px

data = {'children': ['Developmental Evaluation', 'Diagnosed Condition', 'Both'], 
        'number' : ['4065','666','853']}
df = pd.DataFrame(data, columns = ['children', 'number'])
print(df)





In [ ]:
df

In [ ]:
fig = px.pie(df, values='number', names= 'children', title="Number of Children Per Category")
fig.update_traces(textposition = 'outside', textinfo = 'percent+label')
fig.show()

In [ ]:
# percent of children completing program (6+ months)

children = ['Developmental Evaluation', 'Diagnosed Condition', 'Both']
number = [4065,666,853]

#seaborn color palette to plot pie chart 
colors = sns.color_palette('pastel')

#creating pie chart using matplotlib
plt.pie(number, colors=colors, autopct='%.0f%%')
plt.title("Percent of Children Completing Program (6+ months)")

#adding a legend 
plt.legend(labels=children, loc='upper left',
          bbox_to_anchor=(1,0.75), ncol=1)

plt.show()


In [ ]:
# percent of children referred to program 

children = ['Developmental Evaluation', 'Diagnosed Condition', 'Both']
percent = [28317,5298,4675]

#seaborn color palette to plot pie chart 
colors = sns.color_palette('pastel')

#creating pie chart using matplotlib
plt.pie(percent, colors=colors, autopct='%.0f%%')
plt.title("Percent of Children Referred to Program")

#adding a legend 
plt.legend(labels=children, loc='upper left',
          bbox_to_anchor=(1,0.75), ncol=1)

plt.show()


In [ ]:
# percent of children completed vs referred 

children = ['Developmental Evaluation', 'Diagnosed Condition', 'Both']
percent = [0.14355334251509694, 0.12570781426953567, 0.18245989304812835]

#seaborn color palette to plot pie chart 
colors = sns.color_palette('pastel')

#creating pie chart using matplotlib
plt.pie(percent, colors=colors, autopct='%.0f%%')
plt.title("Percent of Children Referred vs Completed Per Category")

#adding a legend 
plt.legend(labels=children, loc='upper left',
          bbox_to_anchor=(1,0.75), ncol=1)

plt.show()

In [ ]:
# #scatter plot by groups 

# #data i am using 
# data = df_merge1


# #creating plot 
# fig = plt.figure()
# ax = fig.add_subplot(1,1,1 axis="1.0")



In [ ]:
df_merge1

In [ ]:
4065/28317*100


In [ ]:
percent = [(4065/28317),(666/5298),(853/4675)]
percent

In [ ]:
data1 = {'Eligibility Category': ['Developmental Evaluation', 'Diagnosed Condition', 'Both'], 
        'Children Completed the Program' : [4065,666,853],
        'Children Referred to Program' :[28317,5298,4675],
         }
df1 = pd.DataFrame(data1, columns = ['Eligibility Category', 'Children Completed the Program','Children Referred to Program'])
print(df1)
df1

In [ ]:
df1

In [ ]:
#just doing some math so i can get the total completion instead of the % on total completion 
28317-4065, 5298-666, 4675-853


In [ ]:
df1.plot( x="Eligibility Category", y=["Children Referred to Program", "Children Completed the Program"], kind="bar");
plt.xticks(rotation=0, fontsize = 10)
plt.xlabel('Eligibility Category', fontweight='bold')
plt.ylabel('Number of Children', fontweight='bold')
plt.title('Referred v Completed', fontweight='bold');

In [ ]:
#adding lables to my bar chart 

labels =['Develeopmental Evaluation', 'Diagnosed Condition', 'Both']
referred =[28317, 5298, 4675]
completed =[4065, 666, 853]

x =np.arange(len(labels)) #the label location
width = 0.35 # the width of the bars


#legend 
fig, ax = plt.subplots(figsize=(12,8))
rects1 = ax.bar(x - width/2, referred, width, label='Number of Children Referred')
rects2 = ax.bar( x + width/2, completed, width, label='Number of Children Completed')

#adding text for labels, title and x-axis tick labels...
ax.set_ylabel('Number of Children')
ax.set_title('Referred v Completed')
ax.set_xticks(x, labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

plt.show()

In [ ]:
#seaborn plotting 
sns.set(style='white')

#creating group bar chart
sns.barplot(x='Eligibility Category', y='Children Referred to Program', data=df1);



In [ ]:
24525+4632+3822

notes:
slide 3 drop pie chart 
slide 4 drop table - we can add data labels onto the bar if we really want to --  keep all of them in slide 4 for(time/days)
slide 5 y axis - number of kids update, let's have the labels all match on the axis -- lets get all of them to be buckets (.head()) for 10
slide 6 dev days, diag days, both days; OR drop first, and then keep middle bar cluster and last bar cluster - donn't need table 
slide 7 - sort values on bar chart from largest to smallest 
slide 8 - fix the titles, pick btw one or the other of the distributions of time 

can sort values 